> IMPORTS


In [3]:
# Manipulação de Arquivos e Sistema
import os
import sys
from tqdm.notebook import tqdm # Versão especial do tqdm para o Jupyter

# Visão Computacional e Imagem
import cv2
import PIL.Image
from PIL import Image

# Machine Learning e Deep Learning (PyTorch)
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Biometria e Detecção Facial
import mediapipe as mp

# Processamento de Dados e Visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Verificação de Versões e Hardware
print(f"Versão do Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"MediaPipe: {mp.__version__}")

Versão do Python: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
PyTorch: 2.5.1
OpenCV: 4.10.0
MediaPipe: 0.10.31


> YOLO dataset training


In [4]:
from ultralytics import YOLO # Para detectar caminhões, prédios, etc.

model_yolo = YOLO('yolov8n.pt') # Modelo leve que identifica 80 tipos de objetos

def analisar_contexto(frame):
    results = model_yolo(frame)
    return results

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\felfe\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


> Teste de video teclado/mouse


In [9]:
import cv2
from ultralytics import YOLO

# Carrega o modelo 
model = YOLO('yolov8n.pt') 

# Caminho do seu vídeo
# Use o caminho exato que aparece no seu VS Code
video_path = 'video_test/teste_perifericos.mp4'

# Abre o vídeo
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo! Verifique o caminho.")

while cap.isOpened():
    success, frame = cap.read()
    
    if success:
        # Roda a detecção (stream=True é bom para vídeos longos)
        results = model(frame, stream=True)
        
        for r in results:
            # r.plot() cria a imagem com as caixinhas e nomes (mouse, keyboard)
            annotated_frame = r.plot() 
            
            # Mostra a janela
            cv2.imshow("IA Detectando Objetos Reais", annotated_frame)

        # O cv2.waitKey(1) é essencial para a janela não travar
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        # Fim do vídeo
        break

cap.release()
cv2.destroyAllWindows()


0: 640x384 1 keyboard, 112.3ms
Speed: 24.0ms preprocess, 112.3ms inference, 15.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 47.0ms
Speed: 2.5ms preprocess, 47.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 43.7ms
Speed: 2.4ms preprocess, 43.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 45.8ms
Speed: 2.3ms preprocess, 45.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 42.6ms
Speed: 2.4ms preprocess, 42.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 43.1ms
Speed: 2.4ms preprocess, 43.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 44.0ms
Speed: 2.3ms preprocess, 44.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 1 cell phone, 45.3ms
Speed: 5.4ms preprocess, 45.3ms inference, 1.1ms postpr